## Imports

In [1]:
import os
import shutil
import json
os.chdir("/Users/atharvas/Documents/FALL_2020/vdp") # point this to the project directory 


## Make sets

In [2]:
vdp_params = object()
with open("./config.json") as f:
    vdp_params = json.load(f)
    

In [3]:
def make_set(params):
    """
    Inputs: 
    vdp_params: dict
        A json file containing formatted dict
    
    Notes:
        dict must be formatted as such:
        params = {
            "name"  : "str", 
            "train" : ["1.jpg", "2.jpg", "3.jpg"],
            "test"  : ["4.jpg", "5.jpg", "6.jpg"]
        }
    """
    #@TODO Input verification
    normalized_name = params["name"].replace(" ", "_").lower()
    del params['name']
    folder_path = os.path.join("./data/interim/", normalized_name)
    #@TODO Logging
    for img_path in params['test']:
        new_img_path = (os.path.join(folder_path, "test", os.path.basename(img_path)))
        os.makedirs(new_img_path, exist_ok=True)
        shutil.copy(img_path, new_img_path)
    del params['test']

    for img_path in params['train']:
        new_img_path = (os.path.join(folder_path, "train", os.path.basename(img_path)))
        os.makedirs(new_img_path, exist_ok=True)
        shutil.copy(img_path, new_img_path)
    del params['train']
    
    if len(params):
        config_path = (os.path.join(folder_path, "config.json"))
        with open(config_path, 'w') as fp:
            json.dump(params, fp)        
    
    


In [5]:
def controller(input_path, output_path, glove_path, model_path, log_path, sg_tools_rel_path="tools/relation_test_net.py", cuda_device_port=0, n_proc=1):
    cmd = f"""CUDA_VISIBLE_DEVICES={cuda_device_port}
    python -m torch.distributed.launch --master_port 10027
    --nproc_per_node={n_proc}
    {sg_tools_rel_path}
    --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml"
    MODEL.ROI_RELATION_HEAD.USE_GT_BOX False
    MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False
    MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor
    MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE 
    MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum 
    MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs 
    TEST.IMS_PER_BATCH 1 
    DTYPE "float16" 
    GLOVE_DIR {glove_path}
    MODEL.PRETRAINED_DETECTOR_CKPT {model_path}
    OUTPUT_DIR {model_path} 
    TEST.CUSTUM_EVAL True
    TEST.CUSTUM_PATH {input_path}
    DETECTED_SGG_DIR {output_path}
    > {log_path}
    """.replace("\n", " ").replace("    ", "")
    print(cmd)
    os.system(cmd)
    
controller("a", "b", "c", "d", "e")

CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py --config-file &quot;configs/e2e_relation_X_101_32_8_FPN_1x.yaml&quot; MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE  MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum  MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  TEST.IMS_PER_BATCH 1  DTYPE &quot;float16&quot;  GLOVE_DIR c MODEL.PRETRAINED_DETECTOR_CKPT d OUTPUT_DIR d  TEST.CUSTUM_EVAL True TEST.CUSTUM_PATH a DETECTED_SGG_DIR b &gt; e 
